# Context

## Quick reminder on last course

XX

## Goal of this course

XX


# Concepts

## Model Evaluation

For regressors, the evaluation of the model is less straightforward and id usually based upon the average residual between the actual data and the model predictions. __Root mean squared error__ (RMSE) and __mean absolute error__ (MAE) are two commonly used examples of these. Once we have a representation of the error produced by the model we can compare that error with the error of the most simple model of the data (usually the mean). This statistic is known as the coefficient of determination or __R-squared__ and represents the part of variability the model managed to get.

# Regression

The first thing to do is to get our data back.

In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
# We put both training target arrays (regression and classification) in the same txt file
# As both target arrays have the same size we just need to split it it two
# and get the right part for the prediction task
target_training = np.split(np.loadtxt('target_training'), 2)[0].flatten()
features_training = pd.read_csv('features_training.csv')

# Same for test data
target_test = np.split(np.loadtxt('target_test'), 2)[0].flatten()
features_test = pd.read_csv('features_test.csv')

In [3]:
# User id as index
features_training = features_training.set_index('user_id')
features_test = features_test.set_index('user_id')

## Baseline forecasts

As mentionned before we should have a baseline to compare the performance of our models with. Here as we are forecasting a quantitative variable, a simple model is for example the one that predicts a constant value like the mean or the median we can observe on our past data. 